In [143]:
import numpy as np
import pandas as pd
import sklearn
import re
#from sklearn.model_selection import train_test_split
from sklearn.cross_validation import train_test_split
from sklearn.linear_model import LogisticRegression

In [144]:
tdf = pd.read_csv("train.csv")
final_test = pd.read_csv("test.csv")
final_test['Survived']=0
tdf = tdf.append(final_test)
#tdf[tdf['PassengerId'] == 891]
#tdf[:891]
#tdf[891:]

In [145]:
#Gender dummy
tdf['male'] = np.where(tdf['Sex'] == 'male',1,0)
tdf['female'] = np.where(tdf['Sex'] == 'female',1,0)

#Ticket class dummy
tdf['Pclass_1'] = np.where(tdf['Pclass'] == 1,1,0)
tdf['Pclass_2'] = np.where(tdf['Pclass'] == 2,1,0)
tdf['Pclass_3'] = np.where(tdf['Pclass'] == 3,1,0)

#Imputing values for embarked
tdf['Embarked'].fillna(value='S',inplace=True)

#Imputing value for Age
tdf['Age'].fillna(value=29,inplace=True)

#Imputing 1 Fare value in test data set
tdf['Fare'].fillna(value=33,inplace=True)

#Embarked dummy
tdf['Embarked_S'] = np.where(tdf['Embarked'] == 'S',1,0)
tdf['Embarked_C'] = np.where(tdf['Embarked'] == 'C',1,0)
tdf['Embarked_Q'] = np.where(tdf['Embarked'] == 'Q',1,0)

In [146]:
tdf.head()

,Age,Cabin,Embarked,Fare,Name,Parch,PassengerId,Pclass,Sex,SibSp,Survived,Ticket,male,female,Pclass_1,Pclass_2,Pclass_3,Embarked_S,Embarked_C,Embarked_Q
0,22.0,NaN,S,7.2500,"Braund, Mr. Owen Harris",0,1,3,male,1,0,A/5 21171,1,0,0,0,1,1,0,0
1,38.0,C85,C,71.2833,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,2,1,female,1,1,PC 17599,0,1,1,0,0,0,1,0
2,26.0,NaN,S,7.9250,"Heikkinen, Miss. Laina",0,3,3,female,0,1,STON/O2. 3101282,0,1,0,0,1,1,0,0
3,35.0,C123,S,53.1000,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,4,1,female,1,1,113803,0,1,1,0,0,1,0,0
4,35.0,NaN,S,8.0500,"Allen, Mr. William Henry",0,5,3,male,0,0,373450,1,0,0,0,1,1,0,0


In [147]:
#Percentage of survival by socio-economic status
#Higher proportion of pessengers from upper class survived than the other two
tdf.loc[tdf['Pclass']==1,'Survived'].value_counts(normalize=True)

0    0.578947
1    0.421053
Name: Survived, dtype: float64

In [148]:
tdf['Fare'].mean()
tdf['Fare'].isnull().value_counts()

False    1309
Name: Fare, dtype: int64

In [149]:
cols=['Age','SibSp','Parch','Fare','male','female','Pclass_1','Pclass_2','Pclass_3','Embarked_S','Embarked_C','Embarked_Q']
X = tdf[:891][cols]
y = tdf[:891]['Survived']

In [150]:
# Train test split --random
#train,test = train_test_split(tdf,test_size=0.25, random_state=44)
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.25, random_state=44)

In [151]:
#Training the model
logreg = LogisticRegression()
logreg.fit(X_train,y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [152]:
y_pred = logreg.predict(X_test)
print(logreg.score(X_test,y_test))

0.748878923767


In [153]:
final_y_pred = logreg.predict(tdf[891:][cols])

In [169]:
finalDF = pd.DataFrame(final_y_pred, tdf[891:]['PassengerId'])
finalDF.rename(columns={0:'Survived'},inplace=True)

In [171]:
finalDF.head()

,Survived
PassengerId,
892,0
893,0
894,0
895,0
896,1


In [173]:
finalDF.to_csv('Pred.csv')